In [2]:
import pandas as pd
import os

In [95]:
def clean_raw_data(df):
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    df.drop(["Id", "Geography"], axis=1, inplace=True)
    cols = df.columns
    cols = [x for x in cols if "Margin of Error" in x]
    df.drop(cols, axis=1, inplace=True)
    df.rename(columns={col: col.replace("Estimate; Total:", "Total") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Estimate: - ", "") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Estimate; ", "") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Id2", "Id") for col in df.columns}, inplace=True)
    df.sort_values("Id", inplace=True)
    return df

def clean_list_raw_data(dfs):
    cleaned_dfs = []
    for df in dfs:
        cleaned_dfs.append(clean_raw_data(df.copy()))
        
    df_length = [x.shape[0] for x in cleaned_dfs]
    assert len(set(df_length)) == 1
    results = pd.concat(cleaned_dfs, axis=1)
    results = results.loc[:,~results.columns.duplicated()]
    return results

def compile_county_data(folder):
    my_data = []
    for file in os.listdir(folder):
        if file.endswith("with_ann.csv"):
            my_data.append(pd.read_csv(os.path.join(folder, file)))
    results = clean_list_raw_data(my_data)
    return results

In [97]:
results = compile_county_data("data/census_data")
results.to_csv("data/census_data.csv")

In [98]:
results

,Id,Median number of rooms,Total,Total - Imputed,Total - Not imputed,Family households:,Family households: - Married-couple family,Family households: - Other family:,"Family households: - Other family: - Male householder, no wife present","Family households: - Other family: - Female householder, no husband present",...,"Enrolled in school: - Enrolled in college, undergraduate years",Enrolled in school: - Graduate or professional school,Not enrolled in school,"Housing units with a mortgage, contract to purchase, or similar debt:","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both:","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both: - Second mortgage only","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both: - Home equity loan only","Housing units with a mortgage, contract to purchase, or similar debt: - Both second mortgage and home equity loan","Housing units with a mortgage, contract to purchase, or similar debt: - No second mortgage and no home equity loan",Housing units without a mortgage
1,360050001000,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,360050001001,-,3987,0,3987,0,0,0,0,0,...,0,0,7703,0,0,0,0,0,0,0
3,360050002000,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,360050002001,5.4,1075,60,1015,380,247,133,23,110,...,74,21,1242,130,8,0,8,0,122,122
5,360050002002,4.8,1426,151,1275,432,278,154,32,122,...,70,11,1760,175,6,0,6,0,169,72
6,360050002003,5.0,725,50,675,311,205,106,75,31,...,107,19,828,151,6,6,0,26,119,46
7,360050004000,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,360050004001,5.1,343,15,328,182,46,136,28,108,...,32,0,440,99,0,0,0,28,71,28
9,360050004002,4.8,832,8,824,397,172,225,41,184,...,51,32,975,166,28,19,9,0,138,71
10,360050004003,5.0,2089,9,2080,826,435,391,148,243,...,213,99,2281,816,63,16,47,0,753,102


In [101]:
len('360050183023000'), len('360050002002')

(12, 12)